# The Third set of parameters

**maxlen = 200,
batch size = 8,
epochs = 10,
steps per epoch = 64,
embedding dimension = 64**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import keras

df=pd.read_csv("/content/drive/MyDrive/Colab Notebooks/SU23 Deep Learning/Final Project/Yelp Restaurant Reviews.csv",index_col=0)
df=df.loc[:,["Review Text","Rating"]].dropna().reset_index(drop=True)

In [3]:
from sklearn.model_selection import train_test_split
rev=df["Review Text"]
target=df["Rating"]
x_train, x_test, y_train, y_test = train_test_split(rev,target,test_size=0.2,random_state=0,stratify=target)

In [4]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

#Text sampling
max_features = 10000
maxlen = 200
batch_size = 8

#Tokenizing
tokenizer=Tokenizer(max_features,oov_token="<OOV>")
tokenizer.fit_on_texts(x_train)
X_train=tokenizer.texts_to_sequences(x_train)
X_test=tokenizer.texts_to_sequences(x_test)
print(len(X_train), 'train sequences')
print(len(X_test), 'test sequences')

#Turns tockenized texts into padded sequences
print('Pad sequences (samples x text length)')
X_train = pad_sequences(X_train, maxlen=maxlen)
X_test = pad_sequences(X_test, maxlen=maxlen)
print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)

15916 train sequences
3980 test sequences
Pad sequences (samples x text length)
X_train shape: (15916, 200)
X_test shape: (3980, 200)


# SimpleRNN

In [5]:
from keras.models import Sequential
from keras.layers import Embedding, SimpleRNN, Dense

model = Sequential()
embedding_dim = 64
model.add(Embedding(max_features, embedding_dim, input_length=maxlen))
model.add(SimpleRNN(64, return_sequences=True))
model.add(SimpleRNN(32, return_sequences=True))
model.add(SimpleRNN(16))
model.add(Dense(1))

model.compile(optimizer='adam', loss='mae', metrics='mape') #mape: Mean Absolute Percentage Error
history = model.fit(X_train, y_train,
                    epochs=10,
                    steps_per_epoch = 64,
                    batch_size=batch_size,
                   validation_data=(X_test,y_test))

keras.backend.clear_session()
del model

Epoch 1/10
64/64 [==============================] - 32s 448ms/step - loss: 1.3891 - mape: 43.2056 - val_loss: 0.9200 - val_mape: 42.5311
Epoch 2/10
64/64 [==============================] - 29s 453ms/step - loss: 0.9048 - mape: 43.1671 - val_loss: 0.8984 - val_mape: 44.2470
Epoch 3/10
64/64 [==============================] - 30s 481ms/step - loss: 0.8832 - mape: 46.6766 - val_loss: 0.8730 - val_mape: 46.2616
Epoch 4/10
64/64 [==============================] - 23s 362ms/step - loss: 0.8719 - mape: 47.4656 - val_loss: 0.8725 - val_mape: 46.2928
Epoch 5/10
64/64 [==============================] - 28s 448ms/step - loss: 0.8311 - mape: 41.8758 - val_loss: 0.8699 - val_mape: 46.5014
Epoch 6/10
64/64 [==============================] - 21s 328ms/step - loss: 0.8702 - mape: 47.2314 - val_loss: 0.8701 - val_mape: 46.4843
Epoch 7/10
64/64 [==============================] - 23s 361ms/step - loss: 0.8904 - mape: 52.1088 - val_loss: 0.8710 - val_mape: 46.4122
Epoch 8/10
64/64 [=======================

# Pure CNN with word Embedding

In [6]:
from keras.models import Sequential
from keras.layers import Embedding
from keras import layers

model = Sequential()
embedding_dim = 64
model.add(Embedding(max_features, embedding_dim, input_length=maxlen))
model.add(layers.Conv1D(64, 5, activation='relu'))
model.add(layers.MaxPooling1D(3))
model.add(layers.Conv1D(32, 5, activation='relu'))
model.add(layers.GlobalMaxPooling1D())
model.add(layers.Dense(1))

model.compile(optimizer='adam', loss='mae', metrics='mape') #mape: Mean Absolute Percentage Error
history = model.fit(X_train, y_train,
                    epochs=10,
                    steps_per_epoch = 64,
                    batch_size=batch_size,
                   validation_data=(X_test,y_test))

keras.backend.clear_session()
del model

Epoch 1/10
64/64 [==============================] - 4s 42ms/step - loss: 1.9824 - mape: 68.8460 - val_loss: 0.9075 - val_mape: 39.4654
Epoch 2/10
64/64 [==============================] - 2s 38ms/step - loss: 0.9558 - mape: 44.2145 - val_loss: 0.8379 - val_mape: 38.5946
Epoch 3/10
64/64 [==============================] - 4s 57ms/step - loss: 0.8399 - mape: 37.3487 - val_loss: 0.8158 - val_mape: 38.9592
Epoch 4/10
64/64 [==============================] - 4s 69ms/step - loss: 0.8632 - mape: 40.7771 - val_loss: 0.8001 - val_mape: 37.0872
Epoch 5/10
64/64 [==============================] - 2s 39ms/step - loss: 0.7926 - mape: 33.5856 - val_loss: 0.8163 - val_mape: 34.1767
Epoch 6/10
64/64 [==============================] - 4s 57ms/step - loss: 0.8552 - mape: 39.9248 - val_loss: 0.7741 - val_mape: 34.1882
Epoch 7/10
64/64 [==============================] - 3s 41ms/step - loss: 0.7868 - mape: 35.4705 - val_loss: 0.7675 - val_mape: 34.7202
Epoch 8/10
64/64 [==============================] - 4s 

# Pure LSTM with Word Embedding

In [7]:
from keras.models import Sequential
from keras.layers import Embedding
from keras import layers

model = Sequential()
embedding_dim = 64
model.add(Embedding(max_features, embedding_dim, input_length=maxlen))
model.add(layers.LSTM(32,dropout=0.2,recurrent_dropout=0.2,return_sequences=True))
model.add(layers.LSTM(16))
model.add(layers.Dense(1))
model.compile(optimizer='adam', loss='mae', metrics='mape')
history = model.fit(X_train, y_train,
                    epochs=10,
                    steps_per_epoch = 64,
                    batch_size=batch_size,
                    validation_data=(X_test,y_test))
keras.backend.clear_session()
del model

Epoch 1/10
64/64 [==============================] - 46s 650ms/step - loss: 2.4295 - mape: 61.4263 - val_loss: 1.0511 - val_mape: 39.7621
Epoch 2/10
64/64 [==============================] - 37s 589ms/step - loss: 0.9553 - mape: 43.8990 - val_loss: 0.9110 - val_mape: 43.2483
Epoch 3/10
64/64 [==============================] - 39s 617ms/step - loss: 0.9054 - mape: 45.8000 - val_loss: 0.8788 - val_mape: 45.7955
Epoch 4/10
64/64 [==============================] - 39s 617ms/step - loss: 0.8520 - mape: 43.7827 - val_loss: 0.8695 - val_mape: 46.5360
Epoch 5/10
64/64 [==============================] - 39s 616ms/step - loss: 0.8890 - mape: 49.1702 - val_loss: 0.8701 - val_mape: 46.4865
Epoch 6/10
64/64 [==============================] - 41s 639ms/step - loss: 0.9353 - mape: 50.7582 - val_loss: 0.8742 - val_mape: 46.1589
Epoch 7/10
64/64 [==============================] - 38s 593ms/step - loss: 0.8893 - mape: 45.5898 - val_loss: 0.8749 - val_mape: 46.0972
Epoch 8/10
64/64 [=======================

# Combine CNN and LSTM with Word Embedding

In [8]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Conv1D, MaxPooling1D
from keras.layers import Dense

model = Sequential()
embedding_dim = 64
model.add(Embedding(max_features, embedding_dim, input_length=maxlen))
model.add(layers.Conv1D(64, 5, activation='relu'))
model.add(layers.MaxPooling1D(3))
model.add(layers.Conv1D(32, 5, activation='relu'))
model.add(layers.LSTM(32,dropout=0.2,recurrent_dropout=0.2,return_sequences=True))
model.add(layers.LSTM(16))
model.add(layers.Dense(1))

model.compile(optimizer='adam', loss='mae', metrics='mape')
history = model.fit(X_train, y_train,
                    epochs=10,
                    steps_per_epoch = 64,
                    batch_size=batch_size,
                   validation_data=(X_test,y_test))
keras.backend.clear_session()
del model

Epoch 1/10
64/64 [==============================] - 22s 234ms/step - loss: 1.9487 - mape: 56.1985 - val_loss: 0.9224 - val_mape: 42.2480
Epoch 2/10
64/64 [==============================] - 16s 258ms/step - loss: 0.7943 - mape: 39.4308 - val_loss: 0.8684 - val_mape: 46.4591
Epoch 3/10
64/64 [==============================] - 17s 262ms/step - loss: 0.7865 - mape: 37.9147 - val_loss: 0.8670 - val_mape: 46.3123
Epoch 4/10
64/64 [==============================] - 14s 225ms/step - loss: 0.8429 - mape: 44.2264 - val_loss: 0.8672 - val_mape: 45.6058
Epoch 5/10
64/64 [==============================] - 14s 223ms/step - loss: 0.9099 - mape: 47.4988 - val_loss: 0.8432 - val_mape: 40.0188
Epoch 6/10
64/64 [==============================] - 14s 223ms/step - loss: 0.8690 - mape: 43.4008 - val_loss: 0.7296 - val_mape: 37.0168
Epoch 7/10
64/64 [==============================] - 14s 224ms/step - loss: 0.7043 - mape: 35.8533 - val_loss: 0.6931 - val_mape: 32.1336
Epoch 8/10
64/64 [=======================

# Combine CNN with Bidirectional LSTM

In [9]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Conv1D, MaxPooling1D
from keras.layers import Dense

model = Sequential()
embedding_dim = 64
model.add(Embedding(max_features, embedding_dim, input_length=maxlen))
model.add(layers.Conv1D(64, 5, activation='relu'))
model.add(layers.MaxPooling1D(3))
model.add(layers.Conv1D(32, 5, activation='relu'))
model.add(layers.Bidirectional(
    layers.LSTM(32,dropout=0.2,recurrent_dropout=0.2,return_sequences=True)))
model.add(layers.Bidirectional(layers.LSTM(16)))
model.add(layers.Dense(1))

model.compile(optimizer='adam', loss='mae', metrics='mape')
history = model.fit(X_train, y_train,
                    epochs=10,
                    steps_per_epoch = 64,
                    batch_size=batch_size,
                   validation_data=(X_test,y_test))
keras.backend.clear_session()
del model

Epoch 1/10
64/64 [==============================] - 36s 390ms/step - loss: 1.5996 - mape: 54.7178 - val_loss: 0.8863 - val_mape: 44.9651
Epoch 2/10
64/64 [==============================] - 22s 344ms/step - loss: 0.8291 - mape: 42.2336 - val_loss: 0.8848 - val_mape: 44.6783
Epoch 3/10
64/64 [==============================] - 32s 500ms/step - loss: 0.8916 - mape: 47.8006 - val_loss: 0.8636 - val_mape: 44.8534
Epoch 4/10
64/64 [==============================] - 32s 503ms/step - loss: 0.8252 - mape: 41.7292 - val_loss: 0.8312 - val_mape: 40.0865
Epoch 5/10
64/64 [==============================] - 32s 505ms/step - loss: 0.8032 - mape: 39.4059 - val_loss: 0.7617 - val_mape: 39.6938
Epoch 6/10
64/64 [==============================] - 23s 362ms/step - loss: 0.7828 - mape: 36.4769 - val_loss: 0.8270 - val_mape: 41.1082
Epoch 7/10
64/64 [==============================] - 24s 373ms/step - loss: 0.7101 - mape: 35.9080 - val_loss: 0.6563 - val_mape: 29.7308
Epoch 8/10
64/64 [=======================